In [1]:
from matplotlib import rcParams
import matplotlib.pyplot as plt
import numpy as np

np.seterr(all='ignore')
rcParams['figure.figsize'] = (14, 6)

#扩展MOD
from funcat import * #FUNCAT的同花顺/通达信指标模块：和：&，或：|，其他基本相同。可以使用指标选股！！！
#from pytdx import * #通达信数据更新模块到QUANTAXIS的mongoDB
from tushare import * #TUSHARE数据连接更新。
from rqalpha import * #RQALPHA本地数据连接，通过 rqalpha update_bundle 8点后更新全部数据库
#from jaqs import * #QunatOS的jaqs客户端在线。
#from abupy import * #ABUPY本地化，数据在线下载更新，数据在线下载在根目录abu下
#from quantaxis import * #加载QUANTAXIS数据库模块，首先需要开启mongoDB+web服务器，数据更新通过通达信服务器。
#from hikyuu import * #HIKYUU本地化，数据库在根目录的stock。通过通达信导入。

#读取/设定数据库时间区域，当前股票，选股时间段。 
import time
import datetime
n1 = (time.strftime("20%y-%m-%d"))## 20yy-mm-dd格式，输出当日日期
print ("读取当日时间格式n1",n1) #当日时间格式：yyyy-mm-dd
n2 = (time.strftime("20%y%m%d"))## 20yymmdd格式，输出当日日期
print ("读取当日时间格式n2",n2) #当日时间格式：yyyymmdd
def getYesterday(): 
    today=datetime.date.today() 
    oneday=datetime.timedelta(days=1) 
    yesterday=today-oneday  
    return yesterday
n3 = (getYesterday())# 输出前一天日期
print ("读取前一日时间格式n3",n3) #前日时间格式：yyyy-mm-dd
i = datetime.datetime.now()#当前日期
n4 = ("%s%s%s" % (i.year, i.month, i.day) )
print ("读取当日时间格式n4",n4) #当日时间格式：yyyymmdd
n5 = ("%s" % i.isoformat() )
print ("读取当日时间格式n5",n5) #当日时间格式：yyyy-mm-dd:...

n8="20180209"#开始时间
print ("手动设定选股开始时间格式n8",n8)

n9="20180209"#结束时间
print ("手动设定选股结束时间格式n9",n9)

n10="20170101"#设定起始时间
print ("手动设定数据更新设定起始时间格式n10",n10)
x1="000001.XSHG"
print ("设定股票代码",x1)

#设定数据库连接：RQALPHA/TUSHARE
from funcat.data.tushare_backend import TushareDataBackend
from funcat.data.rqalpha_data_backend import RQAlphaDataBackend
backend = "TUSHARE"
if backend == "rqalpha":
    set_data_backend(RQAlphaDataBackend("~/.rqalpha/bundle"))
elif backend == "tushare":
    set_data_backend(TushareDataBackend())
set_start_date(n10)

#当前股票最新数据显示。
S(x1)  # 设置当前关注股票
T(n3)   # 设置当前观察日期

print ("打印 开盘 最高 最低 收盘 成交量",O, H, L, C, V)  # 打印 Open High Low Close
print ("当天涨幅",C / C[1] - 1)  # 当天涨幅
print ("打印60日均线",MA(C, 60))  # 打印60日均线
print ("判断收盘价是否大于60日均线",C > MA(C, 60))  # 判断收盘价是否大于60日均线
print ("30日最高价",HHV(H, 30))  # 30日最高价
print ("最近30日，收盘价 Close 大于60日均线的天数",COUNT(C > MA(C, 60), 30))  # 最近30日，收盘价 Close 大于60日均线的天数
print ("10日均线上穿",CROSS(MA(C, 10), MA(C, 20)))  # 10日均线上穿
print ("最近10天收阳线的天数",COUNT(C > O, 10))  # 最近10天收阳线的天数
print ("最近60天K线实体的最高价",HHV(MAX(O, C), 60))  # 最近60天K线实体的最高价
print ("5日均线上穿10日均线",CROSS(MA(C, 5), MA(C, 10)))  # 5日均线上穿10日均线

print ("《选股》读取时间：==》n5",n5)   


#####BOLL选股 ！！！
select(
    lambda : CROSS(L,((MA((O+H+L+C)/4,22))-STD(H,22)*2)) and CROSS(L,(MA((O+H+L+C)/4,22))),
    start_date=(n8), end_date=(n2),
)    

print ("《结束选股》读取结束时间：==》n5",n5) 



读取当日时间格式n1 2018-02-12
读取当日时间格式n2 20180212
读取前一日时间格式n3 2018-02-11
读取当日时间格式n4 2018212
读取当日时间格式n5 2018-02-12T13:39:51.510886
手动设定选股开始时间格式n8 20180205
手动设定选股结束时间格式n9 20180205
手动设定数据更新设定起始时间格式n10 20170101
设定股票代码 000001.XSHG
打印 开盘 最高 最低 收盘 成交量 

FormulaException: DATA UNAVAILABLE

In [2]:
#N=20,P=2 BOLL 布林带
MID = MA((O+H+L+C)/4, 22)
UPPER = MID + STD(L, 22) * 2
LOWER = MID - STD(H, 22) * 2
ax = plt.subplot()
buy_signal = CROSS(L, LOWER) and CROSS(L,MID) 
sell_signal = CROSS(UPPER,H) and CROSS(UPPER,L)
ax.plot(H.series, label="H")
ax.plot(L.series, label="L")
ax.plot(MID.series, label="MID")
ax.plot(UPPER.series, label="UPPER")
ax.plot(LOWER.series, label="LOWER")
ax.set_xlim(0)
plt.plot(np.where(buy_signal.series)[0], L.series[np.where(buy_signal.series)[0]], "^", label="buy", markersize=12, color="red")
plt.plot(np.where(sell_signal.series)[0], H.series[np.where(sell_signal.series)[0]], "v", label="sell", markersize=12, color="green")
plt.legend(loc="best")
plt.show()

FormulaException: series size == 0

In [3]:
#N=20,P=2 BOLL 布林带
MID = (MA((O+H+L+C)/4,22))
UPPER = ((MA((O+H+L+C)/4,22))+STD(L,22)*2)
LOWER = ((MA((O+H+L+C)/4,22))-STD(H,22)*2)


ax = plt.subplot()
buy_signal = CROSS(L, LOWER) and CROSS(L,MID) 

buy_signal = CROSS(L,((MA((O+H+L+C)/4,22))-STD(H,22)*2)) and CROSS(L,(MA((O+H+L+C)/4,22))) 



sell_signal = CROSS(UPPER,H) and CROSS(UPPER,L)
ax.plot(H.series, label="H")
ax.plot(L.series, label="L")
ax.plot(MID.series, label="MID")
ax.plot(UPPER.series, label="UPPER")
ax.plot(LOWER.series, label="LOWER")
ax.set_xlim(0)
plt.plot(np.where(buy_signal.series)[0], L.series[np.where(buy_signal.series)[0]], "^", label="buy", markersize=12, color="red")
plt.plot(np.where(sell_signal.series)[0], H.series[np.where(sell_signal.series)[0]], "v", label="sell", markersize=12, color="green")
plt.legend(loc="best")
plt.show()

FormulaException: series size == 0